# Cell tracking plugin 

Create a plugin to track cells 

1) Select image
2) Segment cells
3) Compute tracks
4) Display tracks interactively

In [1]:
from skimage.io import imread
import numpy as np



In [2]:
IMAGE = imread('https://zenodo.org/records/5206107/files/P31-crop.tif?download=1')
IMAGE_CROP = IMAGE[10:20,-100:,-100:]

In [3]:
def segment_cells(images):
    from cellpose import models
    from skimage.util import img_as_float
    
    masks = []
    for image in images:
        model = models.CellposeModel(gpu=True)
        img = img_as_float(image)
        mask, __, __ = model.eval(image)
        masks.append(mask)
        
    return np.stack(masks)

In [4]:
masks = segment_cells(IMAGE_CROP)



Welcome to CellposeSAM, cellpose v4.0.1! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [5]:
label_image = masks

In [6]:
label_image.shape

(10, 100, 100)

In [7]:
IMAGE.shape

(86, 490, 1024)

In [11]:
import btrack
objects = btrack.utils.segmentation_to_objects(label_image)

[INFO][2025/05/09 05:54:08 PM] Localizing objects from segmentation...
100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 589.72it/s]
[INFO][2025/05/09 05:54:08 PM] Objects are of type: <class 'dict'>
[INFO][2025/05/09 05:54:08 PM] ...Found 96 objects in 10 frames.


In [31]:
from btrack import datasets
config_file = datasets.cell_config()
config_file

'/home/daniel/.cache/btrack-examples/examples/cell_config.json'

In [32]:
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a preconfigured config file
  tracker.configure(config_file)

  # append the objects to be tracked
  tracker.append(objects)
    
  # set the volume
  tracker.volume = ((0, 130), (0, 130)) # if the dataset has multiple z-dimensions, we need an additional bracket

  # track
  tracker.track_interactive(step_size=100)

  # get the tracks in a format for napari visualization
  data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2025/05/09 05:38:02 PM] Loaded btrack: /home/daniel/anaconda3/envs/napari-env/lib/python3.13/site-packages/btrack/libs/libtracker.so
[INFO][2025/05/09 05:38:02 PM] Starting BayesianTracker session
[INFO][2025/05/09 05:38:02 PM] Loading configuration file: /home/daniel/.cache/btrack-examples/examples/cell_config.json
[INFO][2025/05/09 05:38:02 PM] Objects are of type: <class 'list'>
[WARNING][2025/05/09 05:38:02 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2025/05/09 05:38:02 PM] Starting tracking... 
[INFO][2025/05/09 05:38:02 PM] Update using: ['MOTION']
[INFO][2025/05/09 05:38:02 PM] Tracking objects in frames 0 to 86 (of 86)...
[INFO][2025/05/09 05:38:02 PM]  - Timing (Bayesian updates: 0.43ms, Linking: 0.16ms)
[INFO][2025/05/09 05:38:02 PM]  - Probabilities (Link: 0.99902, Lost: 0.98133)
[INFO][2025/05/09 05:38:02 PM] SUCCESS.
[INFO][2025/05/09 05:38:02 PM]  - Found 22481 tracks in 86 frames (in 0.0s)
[INFO][2025/05/09 05:38:02 PM]  - Inserted 0 du

In [10]:
import napari 

viewer = napari.view_image(IMAGE_CROP)
# viewer.add_labels(label_image)
# tracks_layer = viewer.add_tracks(data)

ImportError: cannot import name 'display_errors' from 'pydantic.v1.error_wrappers' (/home/daniel/anaconda3/envs/napari-env/lib/python3.13/site-packages/pydantic/v1/error_wrappers.py)

In [7]:



def track_cells(images):

    @thread_worker
    def _segment_cells():
        return labels
    

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 10.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 20.8 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.1
    Uninstalling pydantic-2.10.1:
      Successfully uninstalled pydantic-2.10.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [btrack]━━━━ 2/4 [cvxopt]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
napari 0.6.0 requires pydantic>=2.2.0, but you have pydantic 1.10.22 which is incompatible.


In [30]:
!pip install --upgrade btrack